In [19]:
import lime
import lime.lime_tabular
import time
import pandas as pd
import numpy as np

df = pd.read_csv('../data/california_housing/california_housing.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df2 = pd.read_csv('../data/california_housing/california_housing.csv')
#nombre de maisons à prendre en compte (20k c'est beaucoup trop)
# quelques condition pour nettoyer la base de données
df = df.loc[df['Population']<10000]
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Seulement San Francisco :
# Latitude > 37,2
# Latitude < 38,07
# Longitude > -122,5
# Longitude < -121,75

df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

save_df = df.copy()

N=200
n = N/len(df)
X_all = df.iloc[:,0:8].copy()
df1 = df.sample(frac=n, random_state=14)
df1 = df1.reset_index(drop=True)
save_df1 = df1.copy()

X = df1.iloc[:,0:8]
Y = df1.iloc[:,9]

X_all = X

8

In [6]:
import sklearn.ensemble
rf = sklearn.ensemble.RandomForestRegressor()
rf.fit(X, Y)
model = rf

In [33]:
import warnings
from numba.core.errors import NumbaDeprecationWarning
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=NumbaDeprecationWarning)
warnings.filterwarnings("ignore")
time_init = time.time()
explainer = lime.lime_tabular.LimeTabularExplainer(np.array(X), feature_names=X.columns, class_names=['price'], verbose=False, mode='regression')
N = len(X)
LIME = pd.DataFrame(np.zeros((N, X.shape[-1])))
l = []
for j in range(N):
    l = []
    exp = explainer.explain_instance(
        X.values[j], model.predict
    )
    l = []
    taille = X.shape[-1]
    for ii in range(taille):
        exp_map = exp.as_map()[0]
        for jj in range(taille):
            if ii == exp_map[jj][0]:
                l.append(exp_map[ii][1])
    LIME.iloc[j] = l
LIME

,0,1,2,3,4,5,6,7
0,-1.892674,-0.350861,0.145851,0.129052,0.042793,-0.020821,-0.020217,0.019616
1,1.383938,0.207879,-0.187753,-0.113898,-0.076283,-0.037356,0.023736,-0.006040
2,-1.881541,0.319094,0.152997,0.129551,-0.048103,0.043464,-0.019650,-0.007695
3,1.382944,-0.284895,0.155168,0.126644,-0.047408,-0.031156,-0.004959,-0.000071
4,0.211923,0.172305,0.107365,0.106964,-0.079293,0.072277,0.027857,-0.022690
...,...,...,...,...,...,...,...,...
195,-0.307774,0.195946,0.129633,0.041113,-0.037351,-0.033249,-0.015703,0.006789
196,0.443950,0.166856,0.145944,-0.138452,0.124855,-0.061029,-0.055932,0.009065
197,1.328023,-0.334802,0.249627,0.114147,0.052809,-0.039467,-0.034416,0.004840
198,-1.863589,0.316509,0.122081,-0.046426,-0.024843,-0.022143,0.009176,-0.008313
